# LangChain：Model

在 LangChain 中，**Model** 是與大語言模型（LLM, Large Language Model）互動的核心層，負責管理所有與「模型推論」相關的流程。它提供統一且擴充性高的介面，讓開發者能以一致的方式調用不同來源的語言模型。

透過 Model 介面，開發者可以整合模型至各類應用情境，例如對話系統、文字生成、摘要、情感分析、工具調用等。LangChain 同時支援多種開源與商業模型，包括 OpenAI、Hugging Face Transformers、Anthropic、Google 等，使得模型串接、調用與管理變得更簡單、更模組化。

In [ ]:
!pip install --upgrade langchain
# 安裝langchain + google or openai
!pip install "langchain[google-genai]"
!pip install "langchain[openai]"

In [ ]:
from langchain.chat_models import init_chat_model
from google.colab import userdata
from rich.pretty import pprint
import os

# 設定API Key
os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

### 初始化模型

In [ ]:
model = init_chat_model("google_genai:gemini-2.5-flash-lite")
model = init_chat_model("gpt-5-mini")

### 呼叫LLM

`invoke`

invoke 方法是用來觸發模型的推理過程，它接收輸入並返回相應的結果。invoke 會將用戶的輸入傳遞給模型，並返回生成的文本結果。

In [ ]:
response = model.invoke('你是誰？')

print("完整回覆內容: ", type(response))
pprint(response)
print("純文字內容：", response.text)

訊息角色

| LangChain 訊息類型   | 角色名稱     | 作用介紹                                           |
|---------------------|-----------------|--------------------------------------------------|
| **SystemMessage**    | **system**       | 設定對話背景或規則，用來告訴模型如何進行對話或角色扮演。這類訊息不會直接顯示給用戶，通常用來調整模型的行為或風格。 |
| **HumanMessage**     | **user**         | 來自用戶的訊息，通常是用戶提問或發出指令，模型需要根據這些訊息生成回應。                   |
| **AIMessage**        | **assistant**     | 模型生成的回應，根據用戶訊息（以及系統設定）生成的答案或建議，通常是用來與用戶進行對話。      |

In [ ]:
from langchain.messages import HumanMessage, AIMessage, SystemMessage

# 輸入完整聊天記錄 (OpenAI格式的字典)
conversation = [
    {"role": "system", "content": "你是一個助理叫做一隻羊，幫使用者把英文翻譯成中文"},
    {"role": "user", "content": "Translate: I love programming."},
]
response = model.invoke(conversation)
print(response.text)

In [ ]:
# 另一種對話格式 (Message物件)
conversation = [
    SystemMessage("你是一個助理叫做一隻羊，幫使用者把英文翻譯成中文"),
    HumanMessage("Translate: I love programming."),
]
response = model.invoke(conversation)
print(response.text)

In [ ]:
# 另一種對話格式(tuple)
conversation = [
    ("system", "你是一個助理叫做一隻羊，幫使用者把英文翻譯成中文"),
    ("human", "Translate: I love programming."),
]
response = model.invoke(conversation)
print(response.text)

## 常見Model參數

| 參數名稱         | 類型        | 說明 |
|------------------|-------------|------|
| `model`     | `str`       | 要使用的模型名稱，例如  `gpt-4o-mini`。|
| `temperature`    | `float`     | 控制模型輸出結果的隨機性。數值越高，越具創造性與多樣性；數值越低，回應則越穩定確定性高|
| `timeout`| `float`| 設定請求逾時秒數|
| `max_tokens`     | `int`       | 限制回傳的最大 token 數。|

In [ ]:
model = init_chat_model(
    "google_genai:gemini-2.5-flash-lite",
    # 額外參數
    temperature=0.7,
    timeout=30,
    max_tokens=1000,
)

#### 練習

利用LLM扮演一位「貓咪心理學家」

目標：
1. 使用 model.invoke() 方法來與 AI 對話

2. 使用 SystemMessage 設定 AI 的角色（專業的貓咪心理學家）

3. 使用 HumanMessage 傳入使用者的提問: `我的貓總是在凌晨三點衝來衝去，怎麼辦？`

延伸練習：
4. 改變 SystemMessage 的內容，試試看不同角色（像是獸醫、貓奴、寵物訓練師）

5. 加入多輪對話，用 list 包住多個 HumanMessage 和 AIMessage

6. 試試看修改model temperature=0.2 和 0.9 看看語氣或風格的差異